In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
import re
import string

In [32]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [33]:
df_fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [34]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [35]:
df_fake["class"] = "Fake"
df_true["class"] = "Real"

In [36]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [37]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis = 0, inplace = True)
    
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [38]:
df_fake.shape, df_true.shape

((23471, 5), (21407, 5))

In [39]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",Fake
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",Fake
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",Fake
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",Fake
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",Fake
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",Fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",Fake
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",Fake


In [40]:
df_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",Real
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",Real
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",Real
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",Real
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",Real
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",Real
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",Real
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",Real
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",Real
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",Real


In [41]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

In [42]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",Fake
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",Fake
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",Fake
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",Fake
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",Fake


In [43]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [44]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [45]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [46]:
df = df.sample(frac = 1)

In [47]:
df.head()

,text,class
3029,WASHINGTON (Reuters) - The chairman of the U.S...,Real
3017,WASHINGTON (Reuters) - White House spokesman S...,Real
7003,A caucus formed by Politico consisting of oper...,Fake
1170,People are scratching their heads at an event ...,Fake
21394,"We are devout Catholics, but my daughter goes ...",Fake


In [48]:
df.reset_index(inplace = True)#this returns index in an orderly fashion starting from 0 to length of data as index.
df.drop(["index"], axis = 1, inplace = True)

In [49]:
df.columns

Index(['text', 'class'], dtype='object')

In [50]:
df.head()

,text,class
0,WASHINGTON (Reuters) - The chairman of the U.S...,Real
1,WASHINGTON (Reuters) - White House spokesman S...,Real
2,A caucus formed by Politico consisting of oper...,Fake
3,People are scratching their heads at an event ...,Fake
4,"We are devout Catholics, but my daughter goes ...",Fake


In [51]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [52]:
df["text"] = df["text"].apply(wordopt)

In [53]:
x = df["text"]
y = df["class"]

In [57]:
type(x)

pandas.core.series.Series

In [58]:
df_merge = pd.concat([df_fake,df_true], axis = 0)
df_merge.to_csv("merge.csv")

In [59]:
dataframe = pd.read_csv('merge.csv')
dataframe.head()

,Unnamed: 0,title,text,subject,date,class
0,0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake


In [60]:
dataframe = dataframe.drop(columns=['Unnamed: 0'])

In [61]:
dataframe.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake


In [62]:
x = dataframe['text']
y = dataframe['class']

In [63]:
x

0        Donald Trump just couldn t wish all Americans ...
1        House Intelligence Committee Chairman Devin Nu...
2        On Friday, it was revealed that former Milwauk...
3        On Christmas day, Donald Trump announced that ...
4        Pope Francis used his annual Christmas Day mes...
                               ...                        
44873    ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...
44874    WASHINGTON (Reuters) - The United States sugge...
44875    WASHINGTON (Reuters) - The United States has d...
44876    ISLAMABAD (Reuters) - Outlining a new strategy...
44877    GENEVA (Reuters) - North Korea and the United ...
Name: text, Length: 44878, dtype: object

In [64]:
y

0        Fake
1        Fake
2        Fake
3        Fake
4        Fake
         ... 
44873    Real
44874    Real
44875    Real
44876    Real
44877    Real
Name: class, Length: 44878, dtype: object

In [66]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [67]:
tfvect = TfidfVectorizer(stop_words='english',max_df=0.7)
tfid_x_train = tfvect.fit_transform(x_train)
tfid_x_test = tfvect.transform(x_test)

 LOGISTIC REGRESSION

In [68]:
LR = LogisticRegression()
LR.fit(tfid_x_train,y_train)

LogisticRegression()

In [69]:
pred_lr=LR.predict(tfid_x_test)

In [70]:
LR.score(tfid_x_test, y_test)

0.9860739750445633

In [71]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

        Fake       0.99      0.99      0.99      4707
        Real       0.98      0.99      0.99      4269

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



Decision Tree Classifier

In [72]:
DT = DecisionTreeClassifier()
DT.fit(tfid_x_train, y_train)

DecisionTreeClassifier()

In [73]:
pred_dt = DT.predict(tfid_x_test)

In [74]:
DT.score(tfid_x_test, y_test)

0.9948752228163993

In [75]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

        Fake       0.99      1.00      1.00      4707
        Real       0.99      0.99      0.99      4269

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



Gradient Boosting Classifier

In [76]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(tfid_x_train, y_train)

GradientBoostingClassifier(random_state=0)

In [77]:
pred_gbc = GBC.predict(tfid_x_test)

In [78]:
GBC.score(tfid_x_test, y_test)

0.9939839572192514

In [79]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

        Fake       1.00      0.99      0.99      4707
        Real       0.99      0.99      0.99      4269

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



Random Forest Classifier

In [80]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(tfid_x_train, y_train)

RandomForestClassifier(random_state=0)

In [81]:
pred_rfc = RFC.predict(tfid_x_test)

In [82]:
RFC.score(tfid_x_test, y_test)

0.9886363636363636

In [83]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

        Fake       0.99      0.99      0.99      4707
        Real       0.99      0.99      0.99      4269

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



In [84]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = DT.predict(vectorized_input_data)
    print(prediction)

In [85]:
fake_news_det('MANCHESTER, N.H./STERLING HEIGHTS, MICHIGAN (Reuters) - As Donald Trump and Hillary Clinton criss-crossed the United States on Sunday in a final, frenzied burst of campaigning, the Republican said he was the â€œlast chanceâ€ to fix a broken country while his Democratic rival said a â€œmoment of reckoningâ€ had arrived. The two candidates in Tuesdayâ€™s presidential election presented starkly different views of the nation and evoked similarly disparate reactions from crowds gathered to hear them speak. In Iowa, Trump said he was the â€œlast chanceâ€ to fix immigration and trade. Outside Minneapolis, he said the Somali refugee population there was a â€œdisaster.â€ Supporters near Pittsburgh booed a song by musician Bruce Springsteen, who is set to campaign with Clinton on Monday night.  Chants of â€œlock her upâ€ came in waves even as news broke that the FBI again said Clinton should not face prosecution for her email practices while secretary of state. As rocker and guitarist Ted Nugent warmed up a Trump rally in a Detroit suburb, he grabbed his crotch. â€œIâ€™ve got your blue state right here,â€ Nugent said, referring to states that typically vote for Democratic candidates, including Michigan. â€œBy the way, my language is much, much cleaner, as you know, than Jay Z,â€ Trump said at the same event.  Trump has criticized crude language in a Friday night concert that rapper Jay Z and his wife, Beyonce, held for Clinton in Cleveland. â€œThe most filthy language youâ€™ve ever heard,â€ Trump said at the Minnesota rally. One of his supporters there wore a t-shirt that said: â€œRope. Tree. Journalist. Some Assembly Required.â€ Clinton began and ended Sunday with renditions of â€œAmerica the Beautiful.â€ The first was at a black church in Philadelphia, the second time performed by folk singer James Taylor at a get-out-the-vote rally in New Hampshire, where the crowd swayed and sang along, linking arms. Clinton walked onto the stage with Khizr Khan, whose son was a slain Muslim U.S. soldier. Khan addressed the Democratic National Convention in July, criticizing Trumpâ€™s proposal to ban Muslims from entering the country and offering to loan him a U.S. Constitution. Some in Manchester held up miniature Constitutions in tribute. â€œIn a race that has been marked by ugly suspicion and insults and attacks of all kinds against immigrants, Muslims and so many others, Mr. Khan, I think, reminded all of us that we are Americans,â€ Clinton said. Clinton called Tuesday a â€œcrossroads electionâ€ and a â€œmoment of reckoning,â€ echoing her earlier statements at a Cleveland rally with Cavaliers player LeBron James. â€œOur core values as Americans are being tested,â€ she said. ')

['Real']


In [86]:
fake_news_det('Patrick Henningsen  21st Century WireRemember when the Obama Administration told the world how it hoped to identify 5,000 reliable non-jihadist  moderate  rebels hanging out in Turkey and Jordan, who might want to fight for Washington in Syria? After all the drama over its infamous  train and equip  program to create their own Arab army in Syria, they want to give it another try.This week, Pentagon officials announced their new plan to train up to 7,000 more  moderate  fighters, but this time the project would take place inside Syria (and to hell with international law).We re told that this was requested by Ankara, and with all NATO allies singing the same hymn   claiming that this new effort will help in securing Turkey s porous border with Syria, or so the story goes. Washington s political cover for this is fashioned from the popular post-Paris theme: to protect civilized Europe from invading hordes and the terrorists who hide among them, as stated in the Wall Street Journal: The program would address part of the tense and long-running dialogue between Washington and Ankara over sealing the border, which Western governments have long complained is an avenue for extremists to leave Syria and travel internationally, The timing of this announcement seemed uncanny against the backdrop of an unlikely ISIS  suicide bomb  terrorist attack, in Turkey, of all places, where the victims just happened to be 14 German tourists, feeding perfectly into Europe s new binary fear set of terrorism and immigration. Shades of GLADIO, undoubtedly (but you won t see western journalists entertaining such conspiracy theories).Our friends at Nation Builders Inc are banking on the global public suffering from Syria fatigue, but now is the time to really pay attention. The west s real agenda for the region is starting to unfold Despite all of US-led Coalition s scandals, lies, epic failures, false flags and official conspiracy theories, the mainstream narrative on Syria persists. It s on our TV screens and in our newspapers constantly   from Los Angeles to London, from Paris to Riyadh, every politician and pundit seems to have an opinion about,  what to do with Syria?  Everyone, except the Syria people  themselves, who like the people of Iraq, Libya, Afghanistan, Yemen and ever other country that s found itself under the cleaver of western interests   are never afforded any say in their own destiny. As is tradition in Washington, London and Paris, the only Syrian voices which count are the ones who have been hand-picked by western technocrats and approved by the Saudi royal family to form their ever-changing  recognized  government-in-exile, the Syrian National Council. Currently, that could be Riad Hijab,  George Sabra or Khaled Khoja. Next week, it could be three others, depending on who has fallen in, or out of favor with Washington or Riyadh.To anyone who was listening close enough early on in the conflict, you would have heard determined members of the western-backed  Syrian Opposition  admitting how they would,  make a deal with the devil if that s what it takes to win power.  Four years later, we now see that is exactly what happened, but whether you believe that this  devil is US-led Coalition, or terrorist armies fighting groups (or both) might be cause for debate. Others will argue over which is the bigger devil   is it the Wahabist-Salafist terrorist brigades like al Nusra Front and Islamic State, or is it the West and its GCC partners?The unfortunate reality is that these two devils are not mutually exclusive. The West and its partners not only created this  terror  crisis, but did so with the intent of unveiling a grand  solution  to the problem.Plans A-ZThe terrorist enclave in Syria is designed for purpose   to implant a hostile metastasis right in the middle of the very territory which neocolonial powers plan to tear away from the barely sovereign nations of Iraq and Syria.Since 2011, the western-led cartel has deployed various levels of international subterfuge in Syria, starting with Plan A: the Arab Spring method   igniting a popular street uprising that might capture the world s attention for 15 minutes   enough to generate some high-quality evocative imagery and a social media storm to carry memes of liberty around the globe, inspiring the people on the streets (and on Twitter) to demand  change . Their cries for freedom would then be backed up by the US State Department, along with its functionaries at Human Rights Watch and Amnesty International America, and other Open Society Institute-funded NGOs and  human rights  organizations. Obviously, this did not work. Next, Plan B: the political opposition suddenly transcended into an armed, paramilitary opposition, followed by brutal violence and killing on both sides. The US and its allies wasted no time, pledging weapons, training, cash and logistical support to the armed opposition. The west set up its own parallel government for Syria abroad, but it failed to gain any traction with the Syria people. Then came Plan C: this  opposition  would be quickly overrun and absorbed into a massive array of mostly foreign jihadi and mercenary soldiers of misfortune and other assorted terrorists. This fed into an already established  War on Terror  western narrative. Now the west had a familiar entry point into the fight. The Washington-London-Paris-Tel-Aviv-Riyadh-Doha-Ankara Axis hoped that these terrorists would do the job of overthrowing the government of Bashar al Assad in Damascus, and collapse the country into a spirally pit of sectarian and tribal chaos. Who could blame them? After all, it worked in Libya in late 2011.Fast forward to 2016, having failed to overthrow Assad and implode the nation-state of Syria, the men and women of Nation Builders Inc and their muscles in Brussels (NATO), are quickly moving to Plan D: take territory. That s not to say plans A through D weren t always in the schematics   they certainly were, and further back than many people would dare to speculate.We now know from Wikileaks cables that this operation goes back to at least 2006, but in reality, it s probably much earlier than that. Taking the long view of history and the audacious building, and rebuilding, of a Euro-centric empire in the Middle East, one could say the process began the day the Sykes-Picot Agreement was signed in 1916. Concealed between those post-colonial lines drawn exactly 100 years ago, was the latent potential for the very conflicts and uncivil wars we are seeing today. If those borders are redrawn once again, you can be certain that they will be drawn by the very same powers involved a century ago. Whatever the arrangement, it will be to weaken, not strengthen the region, and certainly not strengthen any of the present nation states in the region, especially Syria. Proprietary powers will want any new states to be stuck in a perpetual state of neocolonial dependency, with an emphasis on financial and security dependency.Mice and Men21WIRE was one of the first news sources to warn about Washington s initial impulse back in 2014 when it began calling for a  No Fly Zone  in Northern Syria. It was clear that a problem, reaction, solution dialectic was in play, with ISIS being the object of the public s reaction. That plan might have come to fruition in the fall of 2015, but a funny thing happened on the way to the UN General Assembly in New York City. Later that week, Russia announced that it would be inserting itself, and its Air Force, into the Syrian quagmire, and with that, thwarted any master plan for a US-Turkey (NATO) controlled No Fly Zone along the Turkish-Syrian border. Suffice to say, that Russia already knew NATO s next move and acted accordingly, and with purpose.When asked two days ago about the prospect of the Washington DC latest brainchild, Train and Equip 2.0, this time inside Syria, here were my initial thoughts: We now know why, and it has more to do with what s beneath the sand than what s on top.As any good pirate will tell you   if you want to claim your share of the booty, you have to do your share of the killing. The Truth About  Train and Equip The previous  train and equip  program ended with Washington s military brass paraded in front of the Senate Armed Services Committee, before being roundly lambasted for failing to train even a handful of  moderate rebels , this after being awarded $500 million by the US taxpayer to get er done. When originally announced amid much fanfare, the public were told that the US government, in conjunction with the CIA, would be arming and training  moderate  militants in Turkey, Jordan or at a base in Saudi Arabia   before sending them to fight inside Syria. Washington claims that these new trainees were meant to fight ISIS, and not the Syrian government forces, but judging by the unwavering  regime change  rhetoric from the west, it would be highly naive to think that all this was strictly for ISIS.Nine months into the train and equip project, US Army Gen. Lloyd Austin III, acknowledged in Sept. 2015 that they had only managed to get  four or five  on the battlefield by then. Not one member of the media challenged this statement, probably because most were happy with the answer given   one which gave any hawkish critics of the Obama administration the pound of flesh they wanted and served to only intensify more calls before the new year for US  boots on the ground  in Syria. Doves loved it because it seemed to prove that backing proxies doesn t work, and that the best solution was to call it quits. For the sanguine mainstream media, it served-up that all-too familiar government narrative, of a  cock-up, not a cover-up.  This is the media s natural default position. In other words,  Well, it didn t work out, but that s nothing new with our incompetent government. Nothing to see here, move along.  To ask how the money disappeared, or really inquire into how it costs $500 million to train only five men   apparently was way too much work for the media. Not one  journalist  stopped to ask, if only 5 fighters were trained and dispatched, then what happened to the half a billion dollars?The real answer to that question should be self-evident to anyone studying reports on the ground in Syria, and is way too uncomfortable for the US media to stomach   that the US  train and equip  program provided the necessary official cover to allow the prosecution of a dirty war under the table. It wasn t a failure like fall guy Gen. Lloyd Austin will have you believe, it was a success.Judging by the proliferation of heavy weaponry and other lethal arms that have made it into the hands of al Qaeda in Syria (al Nusra Front), ISIS, Jaysh al Islam and others   it s clear that Uncle Sam s generous donation of $500 million has definitely been used to equip, if not train, those  rebels  in Syria.Middle East commentator and analyst Sharmine Narwani explains the West s perennial dilemma when trying to identify who their  moderates  are in Syria: For years, Washington has insisted there are armed  moderate  groups in Syria, but have gone to great lengths to avoid naming these  moderates.  Why? Because if moderates were named and identified, the US would have to be very, very certain that no past, present or future  atrocity video  would surface to prove otherwise. And the US could not guarantee this with any of the groups they have armed, trained or financed in Syria over the past five years. The next  train and equip  round will be more focused, and will be used to secure the following  The map above is from July 2015, but still represents the current military chessboard in Syria.Safe Zone =  Sunnistan Somehow it feels like 1998 all over again, with bad memories of the KLA in Kosovo, imported jihadist fighters in Bosnia, and NATO lording over a newly balkanized region.When you hear US officials like John McCain or clueless Republican presidential candidates like Marco Rubio or Carly Fiorina, all crowing for a  Safe Zone  in northern Syria, this is what they are referring to (see map above). Their  safe zone  just so happens to be in the exact same area where ISIS is currently holding court. It s a  safe zone  alright, but not for refugees and  Syrians fleeing the evil Bashar al Assad    rather, the  safe zone  will be for an array of  rebel  and terrorist fighters, part of a defacto Sunni state-lette in waiting.So the US-led Coalition s  safe zone  is the very Caliphate that Americans are decrying.For anyone requiring proof that such a grand chessboard is in play in Syria, they need look no further than a set of US airstrikes that took place before Christmas. On December 6, 2015,  US fighter bombers struck a Syrian military base located in the village of Ayyash in Deir Ezzor Province, east of Raqqa, which killed 3 Syrian Arab Army (SAA) soldiers and wounded additional others. Naturally, the Pentagon swiftly denied that any such attack took place. What happened after this incident was extremely telling, as explained by global affairs analyst Mike Whitney from Counterpunch Magazine: It s also worth noting, that according to South Front military analysis, the US bombing raid coincided with a  a full-scale ISIS offensive on the villages of Ayyash and Bgelia.  In other words, the US attack provided sufficient air-cover for ISIS terrorists to carry out their ground operations. Was that part of the plan or was it merely a coincidence? So the US air force was coordinating with ISIS boots on the ground, to achieve a common military objective. But it didn t end there. Less than 24 hours after the attack, US warplanes bombed the village of Al-Khan in north-eastern Syria killing 26 Syrian civilians including at least four women and seven children and four women. The message the US military is sending with these lethal attacks is that it wants to control the air-space over east Syria where it plans to remove ISIS and establish a de facto Sunni state consistent with its scheme to break Syria and Iraq into smaller cantons governed by local warlords, Islamic fanatics, and US puppets. Also taking place at the same moment on Dec 6th, was Turkey s quiet little invasion into northern Iraq, sending hundreds of troops and at least 20 tanks into territory north of Mosul, right between the Kurds and ISIS. It should be noted that this illegal incursion by NATO member Turkey was not approved by the Iraqi government in Baghdad, and Iraqi PM Haider al-Abadi called for Turkey to  immediately withdraw its troops.  Judging by Washington s silence on the matter, it s obvious that the US gave Turkey its full blessing. Here, Zero Hedge astutely points out the obvious: Most importantly of all, right on what Al-Araby al-Jadeed claims is the smuggling route for illegal ISIS crude into Turkey from Iraq.  Not surprisingly, Turkey s sudden move also came at the same time that Russia began to publicly expose Turkey s clandestine role in facilitating the ISIS oil trade, as well as launching airstrikes to destroy ISIS oil convoys moving from Syria into Turkey. And then Turkish leader Recep Tayyip Erdo an (photo, left) places his military assets at the very location where stolen ISIS oil makes its way from Iraq into Turkey? Hardly a coincidence.Make no mistake, both ends of NATO s military axle, the United States and Turkey, along with their Saudi-backed boots on the ground in ISIS and al Nusra   have each made critical strategic moves in unison, and with a specific military purpose each time, nudging towards securing territory. Once each key piece is finally in position, all that is required is a new  crisis , either in the region, or in Europe, or in America   to allow the Axis powers to move in quickly and take what is needed to implement the next phase of the game, whether that s creating a Kurdish canton, or a Sunni state-lette. Such moves will rarely be covered by the western international media.To be perfectly clear, the terms  Sunni  and  Shia  are the favored sectarian lexicon used by Nation Builders Inc and its international partners in Riyadh and Tel Aviv. My intention here is not to reduce the Muslim world down to this paradigm, but the architects of instability will use these terms, over and over   in their attempt to both marginalize and dehumanized Muslims for their own selfish ends.What s the  safe zone  for? As the US-led Coalition steps up its 18 month-long Punch  n Judy air operation to supplant ISIS from Raqqa, those western-backed jihadis will later require a safe corridor to flee but still remain connected to their Turkish supply lines. Washington s  safe zone  would also provide a much-needed safe haven for  rebels  who have been sent packing by the Syrian government from towns like Homs and others, as part of a painstaking disarming and reconciliation process between the Assad government and  opposition  rebels who are willing to participate in the program. Once the NATO-GCC Axis has established a mini protectorate inside of Syria, this will provide everything ISIS, al Nusra, Ahrar al-Sham, Turkmens et all will need; a secure launching pad for incursions further inside Syria   which will certainly guarantee a longer, protracted conflict in Syria   and beyond.This little Sunni wedge is also conveniently located next to a large rebel-terrorist contingent in Syria s northwestern Idlib governorate (province). Syria s border with Turkey has been porous from the beginning of the conflict, so the proposed  safe zone  will allow Turkey to maintain this status quo   where it has been continuously allowing the free-flow of weapons, rebels and terrorist fighters alike, along with oil, narcotics and human trafficking   to move freely through its southern border with Syria.If  regime  change  rhetoric is anything to go by, along with turning a blind eye to NATO member Turkey s role in facilitating supply lines and safe passage for al Qaeda and ISIS, then Washington s agenda is fairly transparent here. If they can establish a  internationally recognized  safe zone and a No-Fly Zone on top of it, central planners believe they will have the platform they need to eventually destroy Damascus, install their own western puppet, and then carve up Syria and northern Iraq accordingly. ISIS RAT-LINES: Black market oil has been a key component to carrying on both conflicts in Syria, and Iraq.For the architects of instability in Washington, one crucial benefit of carving out Sunni territory in both northern and eastern Syria is that available oil will help fund all of their paramilitary and terrorist needs   making it sustainable. To guarantee this uninterrupted income stream from black market oil which is presently making its way out of Syria and into Turkey and via Kurdistan, help is required from the Kurdish Regional Government (KRG) and other stakeholders, including the British-Turkish firm Genel Energy, just one firm who is providing the end-run to market for illicit ISIS oil. Ditto for a key mover and shaker in Raqqa s new Rockefeller class, Bilal Erdogan, the son of Turkish president, who according to numerous recent reports has already amassed a small fortune from the lucrative oil-for-terror trade.This is a problem, but one which Washington, London and Paris are not overly concerned with, much less upset about. Iraqi, Kurdish and Turkish opposition officials have accused Turkish governments and the KRG of deliberately allowing some of these smuggling operations to take place.Even as provisional state-lettes and cantons are conjured around it, there is no guarantee that Greater Kurdistan will ever see the light of day if Nation Builders Inc having anything to say bout it. The promise of Greater Kurdistan is as important to neocolonialists, than the reality of a Greater Kurdistan. Turkey also has its own interests and ambitions, and a Greater Kurdistan is not compatible with them. Iraq is not crazy about the idea either, because of obvious financial and energy resource reasons. So they remain in limb indefinitely, even as smaller quasi-Kurdish entities sprout up around the KRG. Turkey has its own ongoing  Kurdish problem  within Turkish borders, so Ankara will naturally view any expressions of Kurdish autonomy around it as a threat and potential inspiration for international Kurdish solidarity. Ruling over the Kurds means keeping them divided, and playing them off against their neighbors.There are those within the Turkish establishment who believe there are old scores still unsettled   in reclaiming Turk territory in both Syria and Iraq.SEE ALSO: SPECIAL REPORT: ISIS Oil? Follow the Money (Back to Europe)Here is where Sunnistan proper can emerge, coming into its own once set pieces are in place in Iraq and across Kurdistan, as Mike Whitney explained: So it looks like an agreement has been struck between Turkey, the KRG and the United States to seize parts of northern Iraq and eastern Syria to create a de facto Sunni state that will be jointly-controlled by Ankara and Washington.  It also looks like Obama has agreed to use dodgy jihadi-proxies (aka Terrorists) to work alongside US Special Forces to carry out future military operations.  So while the effort to remove Assad has been temporarily put on the backburner, the determination to destroy Syria is as strong as ever.  Another dismembered part of old Syria and Iraq which central planners are hoping to hack-off is an area called Rojava (see map above), or  Western Kurdistan . This region in northern Syria, formerly known as Syrian Kurdistan, gained its autonomy in November 2013 as part of the ongoing Rojava Revolution. The Kurdish defense forces here are known as the YPG (People s Defense Units) and are not on friendly footing with neighboring Turkey, for a number of reasons, not least of all because YPG defense forces present resistance to both al Nusra and ISIS encroachments into all three Rojava cantons. For Turkey, this is bad for business. How Rojava will factor into Washington s plans is not clear yet, but it occupies the most strategic squares on the chessboard.Make no mistake, the history s actors are all in for this plan. Do not be fooled by talk of  peace and stability  or humanitarian platitudes from western leaders, the UN, or the fair weather international community.Another indication that these plans have the full backing of the neoconservative illiterati, and Israel too, was PNAC walrus and former US Ambassador to the UN John Bolton s recent article in the establishment s daily journal of political posterity, the New York Times, where he states: Today s reality is that Iraq and Syria as we have known them are gone. The Islamic State has carved out a new entity from the post-Ottoman Empire settlement, mobilizing Sunni opposition to the regime of President Bashar al-Assad and the Iran-dominated government of Iraq. Also emerging, after years of effort, is a de facto independent Kurdistan. Central planners will happily frame this geopolitical scenario within the preferred theme of Sunni vs Shia  sectarian strife  in the region, which just happens to suit the interests and desires of both Saudi Arabia and Israel. Bolton confirms this: If, in this context, defeating the Islamic State means restoring to power Mr. Assad in Syria and Iran s puppets in Iraq, that outcome is neither feasible nor desirable. Rather than striving to recreate the post-World War I map, Washington should recognize the new geopolitics. The best alternative to the Islamic State in northeastern Syria and western Iraq is a new, independent Sunni state. Not so fast Mr. Ambassador. The Islamic State is the Sunni State.So for all intents and purposes, he s calling for the establishment of the Caliphate (funny how this goes right over the heads of  conservatives ). This is just one of many geostrategic moves which will guarantee instability and military conflict in the region for another 100 years, as well as provide a timely entr e through to Azerbaijan and into Iran, and later towards the Eurasian heartland of Dagestan (Russia s emerging energy center and geographic underbelly).In the meantime, expect to be pelted with endless propaganda about starving children in one town or another, changing week to week   with the usual slogans like,  we must act now! , and blaming Bashar al Assad for the suffering of his people. It s already wearing thin.Until the US and its NATO-GCC Axis stops flooding Syria and the region with weapons and terrorist fighters, and continues its policy of toppling secular nation-states   then any attempt at holding peace talks will end up back at square one, and stand as another exercise in futility.If the Syrian conflict has been good for one thing (and that s not to say it has), it s that the scripted western narrative, in all of its variations, has finally hit the wall of truth.***Author Patrick Henningsen is a writer, lecturer and global affairs analyst for international TV and radio. He is also the founder and co-editor of 21stCenturyWire.com.READ MORE SYRIA NEWS AT: 21st Century Wire Syria Files')

['Fake']


In [87]:
pickle.dump(DT,open('model.pkl', 'wb'))

In [88]:
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [89]:
def fake_news_det1(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    print(prediction)

In [90]:
fake_news_det1('MANCHESTER, N.H./STERLING HEIGHTS, MICHIGAN (Reuters) - As Donald Trump and Hillary Clinton criss-crossed the United States on Sunday in a final, frenzied burst of campaigning, the Republican said he was the â€œlast chanceâ€ to fix a broken country while his Democratic rival said a â€œmoment of reckoningâ€ had arrived. The two candidates in Tuesdayâ€™s presidential election presented starkly different views of the nation and evoked similarly disparate reactions from crowds gathered to hear them speak. In Iowa, Trump said he was the â€œlast chanceâ€ to fix immigration and trade. Outside Minneapolis, he said the Somali refugee population there was a â€œdisaster.â€ Supporters near Pittsburgh booed a song by musician Bruce Springsteen, who is set to campaign with Clinton on Monday night.  Chants of â€œlock her upâ€ came in waves even as news broke that the FBI again said Clinton should not face prosecution for her email practices while secretary of state. As rocker and guitarist Ted Nugent warmed up a Trump rally in a Detroit suburb, he grabbed his crotch. â€œIâ€™ve got your blue state right here,â€ Nugent said, referring to states that typically vote for Democratic candidates, including Michigan. â€œBy the way, my language is much, much cleaner, as you know, than Jay Z,â€ Trump said at the same event.  Trump has criticized crude language in a Friday night concert that rapper Jay Z and his wife, Beyonce, held for Clinton in Cleveland. â€œThe most filthy language youâ€™ve ever heard,â€ Trump said at the Minnesota rally. One of his supporters there wore a t-shirt that said: â€œRope. Tree. Journalist. Some Assembly Required.â€ Clinton began and ended Sunday with renditions of â€œAmerica the Beautiful.â€ The first was at a black church in Philadelphia, the second time performed by folk singer James Taylor at a get-out-the-vote rally in New Hampshire, where the crowd swayed and sang along, linking arms. Clinton walked onto the stage with Khizr Khan, whose son was a slain Muslim U.S. soldier. Khan addressed the Democratic National Convention in July, criticizing Trumpâ€™s proposal to ban Muslims from entering the country and offering to loan him a U.S. Constitution. Some in Manchester held up miniature Constitutions in tribute. â€œIn a race that has been marked by ugly suspicion and insults and attacks of all kinds against immigrants, Muslims and so many others, Mr. Khan, I think, reminded all of us that we are Americans,â€ Clinton said. Clinton called Tuesday a â€œcrossroads electionâ€ and a â€œmoment of reckoning,â€ echoing her earlier statements at a Cleveland rally with Cavaliers player LeBron James. â€œOur core values as Americans are being tested,â€ she said. ')

['Real']


In [91]:
fake_news_det1('Patrick Henningsen  21st Century WireRemember when the Obama Administration told the world how it hoped to identify 5,000 reliable non-jihadist  moderate  rebels hanging out in Turkey and Jordan, who might want to fight for Washington in Syria? After all the drama over its infamous  train and equip  program to create their own Arab army in Syria, they want to give it another try.This week, Pentagon officials announced their new plan to train up to 7,000 more  moderate  fighters, but this time the project would take place inside Syria (and to hell with international law).We re told that this was requested by Ankara, and with all NATO allies singing the same hymn   claiming that this new effort will help in securing Turkey s porous border with Syria, or so the story goes. Washington s political cover for this is fashioned from the popular post-Paris theme: to protect civilized Europe from invading hordes and the terrorists who hide among them, as stated in the Wall Street Journal: The program would address part of the tense and long-running dialogue between Washington and Ankara over sealing the border, which Western governments have long complained is an avenue for extremists to leave Syria and travel internationally, The timing of this announcement seemed uncanny against the backdrop of an unlikely ISIS  suicide bomb  terrorist attack, in Turkey, of all places, where the victims just happened to be 14 German tourists, feeding perfectly into Europe s new binary fear set of terrorism and immigration. Shades of GLADIO, undoubtedly (but you won t see western journalists entertaining such conspiracy theories).Our friends at Nation Builders Inc are banking on the global public suffering from Syria fatigue, but now is the time to really pay attention. The west s real agenda for the region is starting to unfold Despite all of US-led Coalition s scandals, lies, epic failures, false flags and official conspiracy theories, the mainstream narrative on Syria persists. It s on our TV screens and in our newspapers constantly   from Los Angeles to London, from Paris to Riyadh, every politician and pundit seems to have an opinion about,  what to do with Syria?  Everyone, except the Syria people  themselves, who like the people of Iraq, Libya, Afghanistan, Yemen and ever other country that s found itself under the cleaver of western interests   are never afforded any say in their own destiny. As is tradition in Washington, London and Paris, the only Syrian voices which count are the ones who have been hand-picked by western technocrats and approved by the Saudi royal family to form their ever-changing  recognized  government-in-exile, the Syrian National Council. Currently, that could be Riad Hijab,  George Sabra or Khaled Khoja. Next week, it could be three others, depending on who has fallen in, or out of favor with Washington or Riyadh.To anyone who was listening close enough early on in the conflict, you would have heard determined members of the western-backed  Syrian Opposition  admitting how they would,  make a deal with the devil if that s what it takes to win power.  Four years later, we now see that is exactly what happened, but whether you believe that this  devil is US-led Coalition, or terrorist armies fighting groups (or both) might be cause for debate. Others will argue over which is the bigger devil   is it the Wahabist-Salafist terrorist brigades like al Nusra Front and Islamic State, or is it the West and its GCC partners?The unfortunate reality is that these two devils are not mutually exclusive. The West and its partners not only created this  terror  crisis, but did so with the intent of unveiling a grand  solution  to the problem.Plans A-ZThe terrorist enclave in Syria is designed for purpose   to implant a hostile metastasis right in the middle of the very territory which neocolonial powers plan to tear away from the barely sovereign nations of Iraq and Syria.Since 2011, the western-led cartel has deployed various levels of international subterfuge in Syria, starting with Plan A: the Arab Spring method   igniting a popular street uprising that might capture the world s attention for 15 minutes   enough to generate some high-quality evocative imagery and a social media storm to carry memes of liberty around the globe, inspiring the people on the streets (and on Twitter) to demand  change . Their cries for freedom would then be backed up by the US State Department, along with its functionaries at Human Rights Watch and Amnesty International America, and other Open Society Institute-funded NGOs and  human rights  organizations. Obviously, this did not work. Next, Plan B: the political opposition suddenly transcended into an armed, paramilitary opposition, followed by brutal violence and killing on both sides. The US and its allies wasted no time, pledging weapons, training, cash and logistical support to the armed opposition. The west set up its own parallel government for Syria abroad, but it failed to gain any traction with the Syria people. Then came Plan C: this  opposition  would be quickly overrun and absorbed into a massive array of mostly foreign jihadi and mercenary soldiers of misfortune and other assorted terrorists. This fed into an already established  War on Terror  western narrative. Now the west had a familiar entry point into the fight. The Washington-London-Paris-Tel-Aviv-Riyadh-Doha-Ankara Axis hoped that these terrorists would do the job of overthrowing the government of Bashar al Assad in Damascus, and collapse the country into a spirally pit of sectarian and tribal chaos. Who could blame them? After all, it worked in Libya in late 2011.Fast forward to 2016, having failed to overthrow Assad and implode the nation-state of Syria, the men and women of Nation Builders Inc and their muscles in Brussels (NATO), are quickly moving to Plan D: take territory. That s not to say plans A through D weren t always in the schematics   they certainly were, and further back than many people would dare to speculate.We now know from Wikileaks cables that this operation goes back to at least 2006, but in reality, it s probably much earlier than that. Taking the long view of history and the audacious building, and rebuilding, of a Euro-centric empire in the Middle East, one could say the process began the day the Sykes-Picot Agreement was signed in 1916. Concealed between those post-colonial lines drawn exactly 100 years ago, was the latent potential for the very conflicts and uncivil wars we are seeing today. If those borders are redrawn once again, you can be certain that they will be drawn by the very same powers involved a century ago. Whatever the arrangement, it will be to weaken, not strengthen the region, and certainly not strengthen any of the present nation states in the region, especially Syria. Proprietary powers will want any new states to be stuck in a perpetual state of neocolonial dependency, with an emphasis on financial and security dependency.Mice and Men21WIRE was one of the first news sources to warn about Washington s initial impulse back in 2014 when it began calling for a  No Fly Zone  in Northern Syria. It was clear that a problem, reaction, solution dialectic was in play, with ISIS being the object of the public s reaction. That plan might have come to fruition in the fall of 2015, but a funny thing happened on the way to the UN General Assembly in New York City. Later that week, Russia announced that it would be inserting itself, and its Air Force, into the Syrian quagmire, and with that, thwarted any master plan for a US-Turkey (NATO) controlled No Fly Zone along the Turkish-Syrian border. Suffice to say, that Russia already knew NATO s next move and acted accordingly, and with purpose.When asked two days ago about the prospect of the Washington DC latest brainchild, Train and Equip 2.0, this time inside Syria, here were my initial thoughts: We now know why, and it has more to do with what s beneath the sand than what s on top.As any good pirate will tell you   if you want to claim your share of the booty, you have to do your share of the killing. The Truth About  Train and Equip The previous  train and equip  program ended with Washington s military brass paraded in front of the Senate Armed Services Committee, before being roundly lambasted for failing to train even a handful of  moderate rebels , this after being awarded $500 million by the US taxpayer to get er done. When originally announced amid much fanfare, the public were told that the US government, in conjunction with the CIA, would be arming and training  moderate  militants in Turkey, Jordan or at a base in Saudi Arabia   before sending them to fight inside Syria. Washington claims that these new trainees were meant to fight ISIS, and not the Syrian government forces, but judging by the unwavering  regime change  rhetoric from the west, it would be highly naive to think that all this was strictly for ISIS.Nine months into the train and equip project, US Army Gen. Lloyd Austin III, acknowledged in Sept. 2015 that they had only managed to get  four or five  on the battlefield by then. Not one member of the media challenged this statement, probably because most were happy with the answer given   one which gave any hawkish critics of the Obama administration the pound of flesh they wanted and served to only intensify more calls before the new year for US  boots on the ground  in Syria. Doves loved it because it seemed to prove that backing proxies doesn t work, and that the best solution was to call it quits. For the sanguine mainstream media, it served-up that all-too familiar government narrative, of a  cock-up, not a cover-up.  This is the media s natural default position. In other words,  Well, it didn t work out, but that s nothing new with our incompetent government. Nothing to see here, move along.  To ask how the money disappeared, or really inquire into how it costs $500 million to train only five men   apparently was way too much work for the media. Not one  journalist  stopped to ask, if only 5 fighters were trained and dispatched, then what happened to the half a billion dollars?The real answer to that question should be self-evident to anyone studying reports on the ground in Syria, and is way too uncomfortable for the US media to stomach   that the US  train and equip  program provided the necessary official cover to allow the prosecution of a dirty war under the table. It wasn t a failure like fall guy Gen. Lloyd Austin will have you believe, it was a success.Judging by the proliferation of heavy weaponry and other lethal arms that have made it into the hands of al Qaeda in Syria (al Nusra Front), ISIS, Jaysh al Islam and others   it s clear that Uncle Sam s generous donation of $500 million has definitely been used to equip, if not train, those  rebels  in Syria.Middle East commentator and analyst Sharmine Narwani explains the West s perennial dilemma when trying to identify who their  moderates  are in Syria: For years, Washington has insisted there are armed  moderate  groups in Syria, but have gone to great lengths to avoid naming these  moderates.  Why? Because if moderates were named and identified, the US would have to be very, very certain that no past, present or future  atrocity video  would surface to prove otherwise. And the US could not guarantee this with any of the groups they have armed, trained or financed in Syria over the past five years. The next  train and equip  round will be more focused, and will be used to secure the following  The map above is from July 2015, but still represents the current military chessboard in Syria.Safe Zone =  Sunnistan Somehow it feels like 1998 all over again, with bad memories of the KLA in Kosovo, imported jihadist fighters in Bosnia, and NATO lording over a newly balkanized region.When you hear US officials like John McCain or clueless Republican presidential candidates like Marco Rubio or Carly Fiorina, all crowing for a  Safe Zone  in northern Syria, this is what they are referring to (see map above). Their  safe zone  just so happens to be in the exact same area where ISIS is currently holding court. It s a  safe zone  alright, but not for refugees and  Syrians fleeing the evil Bashar al Assad    rather, the  safe zone  will be for an array of  rebel  and terrorist fighters, part of a defacto Sunni state-lette in waiting.So the US-led Coalition s  safe zone  is the very Caliphate that Americans are decrying.For anyone requiring proof that such a grand chessboard is in play in Syria, they need look no further than a set of US airstrikes that took place before Christmas. On December 6, 2015,  US fighter bombers struck a Syrian military base located in the village of Ayyash in Deir Ezzor Province, east of Raqqa, which killed 3 Syrian Arab Army (SAA) soldiers and wounded additional others. Naturally, the Pentagon swiftly denied that any such attack took place. What happened after this incident was extremely telling, as explained by global affairs analyst Mike Whitney from Counterpunch Magazine: It s also worth noting, that according to South Front military analysis, the US bombing raid coincided with a  a full-scale ISIS offensive on the villages of Ayyash and Bgelia.  In other words, the US attack provided sufficient air-cover for ISIS terrorists to carry out their ground operations. Was that part of the plan or was it merely a coincidence? So the US air force was coordinating with ISIS boots on the ground, to achieve a common military objective. But it didn t end there. Less than 24 hours after the attack, US warplanes bombed the village of Al-Khan in north-eastern Syria killing 26 Syrian civilians including at least four women and seven children and four women. The message the US military is sending with these lethal attacks is that it wants to control the air-space over east Syria where it plans to remove ISIS and establish a de facto Sunni state consistent with its scheme to break Syria and Iraq into smaller cantons governed by local warlords, Islamic fanatics, and US puppets. Also taking place at the same moment on Dec 6th, was Turkey s quiet little invasion into northern Iraq, sending hundreds of troops and at least 20 tanks into territory north of Mosul, right between the Kurds and ISIS. It should be noted that this illegal incursion by NATO member Turkey was not approved by the Iraqi government in Baghdad, and Iraqi PM Haider al-Abadi called for Turkey to  immediately withdraw its troops.  Judging by Washington s silence on the matter, it s obvious that the US gave Turkey its full blessing. Here, Zero Hedge astutely points out the obvious: Most importantly of all, right on what Al-Araby al-Jadeed claims is the smuggling route for illegal ISIS crude into Turkey from Iraq.  Not surprisingly, Turkey s sudden move also came at the same time that Russia began to publicly expose Turkey s clandestine role in facilitating the ISIS oil trade, as well as launching airstrikes to destroy ISIS oil convoys moving from Syria into Turkey. And then Turkish leader Recep Tayyip Erdo an (photo, left) places his military assets at the very location where stolen ISIS oil makes its way from Iraq into Turkey? Hardly a coincidence.Make no mistake, both ends of NATO s military axle, the United States and Turkey, along with their Saudi-backed boots on the ground in ISIS and al Nusra   have each made critical strategic moves in unison, and with a specific military purpose each time, nudging towards securing territory. Once each key piece is finally in position, all that is required is a new  crisis , either in the region, or in Europe, or in America   to allow the Axis powers to move in quickly and take what is needed to implement the next phase of the game, whether that s creating a Kurdish canton, or a Sunni state-lette. Such moves will rarely be covered by the western international media.To be perfectly clear, the terms  Sunni  and  Shia  are the favored sectarian lexicon used by Nation Builders Inc and its international partners in Riyadh and Tel Aviv. My intention here is not to reduce the Muslim world down to this paradigm, but the architects of instability will use these terms, over and over   in their attempt to both marginalize and dehumanized Muslims for their own selfish ends.What s the  safe zone  for? As the US-led Coalition steps up its 18 month-long Punch  n Judy air operation to supplant ISIS from Raqqa, those western-backed jihadis will later require a safe corridor to flee but still remain connected to their Turkish supply lines. Washington s  safe zone  would also provide a much-needed safe haven for  rebels  who have been sent packing by the Syrian government from towns like Homs and others, as part of a painstaking disarming and reconciliation process between the Assad government and  opposition  rebels who are willing to participate in the program. Once the NATO-GCC Axis has established a mini protectorate inside of Syria, this will provide everything ISIS, al Nusra, Ahrar al-Sham, Turkmens et all will need; a secure launching pad for incursions further inside Syria   which will certainly guarantee a longer, protracted conflict in Syria   and beyond.This little Sunni wedge is also conveniently located next to a large rebel-terrorist contingent in Syria s northwestern Idlib governorate (province). Syria s border with Turkey has been porous from the beginning of the conflict, so the proposed  safe zone  will allow Turkey to maintain this status quo   where it has been continuously allowing the free-flow of weapons, rebels and terrorist fighters alike, along with oil, narcotics and human trafficking   to move freely through its southern border with Syria.If  regime  change  rhetoric is anything to go by, along with turning a blind eye to NATO member Turkey s role in facilitating supply lines and safe passage for al Qaeda and ISIS, then Washington s agenda is fairly transparent here. If they can establish a  internationally recognized  safe zone and a No-Fly Zone on top of it, central planners believe they will have the platform they need to eventually destroy Damascus, install their own western puppet, and then carve up Syria and northern Iraq accordingly. ISIS RAT-LINES: Black market oil has been a key component to carrying on both conflicts in Syria, and Iraq.For the architects of instability in Washington, one crucial benefit of carving out Sunni territory in both northern and eastern Syria is that available oil will help fund all of their paramilitary and terrorist needs   making it sustainable. To guarantee this uninterrupted income stream from black market oil which is presently making its way out of Syria and into Turkey and via Kurdistan, help is required from the Kurdish Regional Government (KRG) and other stakeholders, including the British-Turkish firm Genel Energy, just one firm who is providing the end-run to market for illicit ISIS oil. Ditto for a key mover and shaker in Raqqa s new Rockefeller class, Bilal Erdogan, the son of Turkish president, who according to numerous recent reports has already amassed a small fortune from the lucrative oil-for-terror trade.This is a problem, but one which Washington, London and Paris are not overly concerned with, much less upset about. Iraqi, Kurdish and Turkish opposition officials have accused Turkish governments and the KRG of deliberately allowing some of these smuggling operations to take place.Even as provisional state-lettes and cantons are conjured around it, there is no guarantee that Greater Kurdistan will ever see the light of day if Nation Builders Inc having anything to say bout it. The promise of Greater Kurdistan is as important to neocolonialists, than the reality of a Greater Kurdistan. Turkey also has its own interests and ambitions, and a Greater Kurdistan is not compatible with them. Iraq is not crazy about the idea either, because of obvious financial and energy resource reasons. So they remain in limb indefinitely, even as smaller quasi-Kurdish entities sprout up around the KRG. Turkey has its own ongoing  Kurdish problem  within Turkish borders, so Ankara will naturally view any expressions of Kurdish autonomy around it as a threat and potential inspiration for international Kurdish solidarity. Ruling over the Kurds means keeping them divided, and playing them off against their neighbors.There are those within the Turkish establishment who believe there are old scores still unsettled   in reclaiming Turk territory in both Syria and Iraq.SEE ALSO: SPECIAL REPORT: ISIS Oil? Follow the Money (Back to Europe)Here is where Sunnistan proper can emerge, coming into its own once set pieces are in place in Iraq and across Kurdistan, as Mike Whitney explained: So it looks like an agreement has been struck between Turkey, the KRG and the United States to seize parts of northern Iraq and eastern Syria to create a de facto Sunni state that will be jointly-controlled by Ankara and Washington.  It also looks like Obama has agreed to use dodgy jihadi-proxies (aka Terrorists) to work alongside US Special Forces to carry out future military operations.  So while the effort to remove Assad has been temporarily put on the backburner, the determination to destroy Syria is as strong as ever.  Another dismembered part of old Syria and Iraq which central planners are hoping to hack-off is an area called Rojava (see map above), or  Western Kurdistan . This region in northern Syria, formerly known as Syrian Kurdistan, gained its autonomy in November 2013 as part of the ongoing Rojava Revolution. The Kurdish defense forces here are known as the YPG (People s Defense Units) and are not on friendly footing with neighboring Turkey, for a number of reasons, not least of all because YPG defense forces present resistance to both al Nusra and ISIS encroachments into all three Rojava cantons. For Turkey, this is bad for business. How Rojava will factor into Washington s plans is not clear yet, but it occupies the most strategic squares on the chessboard.Make no mistake, the history s actors are all in for this plan. Do not be fooled by talk of  peace and stability  or humanitarian platitudes from western leaders, the UN, or the fair weather international community.Another indication that these plans have the full backing of the neoconservative illiterati, and Israel too, was PNAC walrus and former US Ambassador to the UN John Bolton s recent article in the establishment s daily journal of political posterity, the New York Times, where he states: Today s reality is that Iraq and Syria as we have known them are gone. The Islamic State has carved out a new entity from the post-Ottoman Empire settlement, mobilizing Sunni opposition to the regime of President Bashar al-Assad and the Iran-dominated government of Iraq. Also emerging, after years of effort, is a de facto independent Kurdistan. Central planners will happily frame this geopolitical scenario within the preferred theme of Sunni vs Shia  sectarian strife  in the region, which just happens to suit the interests and desires of both Saudi Arabia and Israel. Bolton confirms this: If, in this context, defeating the Islamic State means restoring to power Mr. Assad in Syria and Iran s puppets in Iraq, that outcome is neither feasible nor desirable. Rather than striving to recreate the post-World War I map, Washington should recognize the new geopolitics. The best alternative to the Islamic State in northeastern Syria and western Iraq is a new, independent Sunni state. Not so fast Mr. Ambassador. The Islamic State is the Sunni State.So for all intents and purposes, he s calling for the establishment of the Caliphate (funny how this goes right over the heads of  conservatives ). This is just one of many geostrategic moves which will guarantee instability and military conflict in the region for another 100 years, as well as provide a timely entr e through to Azerbaijan and into Iran, and later towards the Eurasian heartland of Dagestan (Russia s emerging energy center and geographic underbelly).In the meantime, expect to be pelted with endless propaganda about starving children in one town or another, changing week to week   with the usual slogans like,  we must act now! , and blaming Bashar al Assad for the suffering of his people. It s already wearing thin.Until the US and its NATO-GCC Axis stops flooding Syria and the region with weapons and terrorist fighters, and continues its policy of toppling secular nation-states   then any attempt at holding peace talks will end up back at square one, and stand as another exercise in futility.If the Syrian conflict has been good for one thing (and that s not to say it has), it s that the scripted western narrative, in all of its variations, has finally hit the wall of truth.***Author Patrick Henningsen is a writer, lecturer and global affairs analyst for international TV and radio. He is also the founder and co-editor of 21stCenturyWire.com.READ MORE SYRIA NEWS AT: 21st Century Wire Syria Files')

['Fake']


In [92]:
confusion_matrix(pred_dt, y_test)

array([[4685,   24],
       [  22, 4245]], dtype=int64)

In [93]:
confusion_matrix(pred_lr, y_test)

array([[4641,   59],
       [  66, 4210]], dtype=int64)

In [94]:
confusion_matrix(pred_rfc, y_test)

array([[4650,   45],
       [  57, 4224]], dtype=int64)

In [95]:
confusion_matrix(pred_gbc, y_test)

array([[4675,   22],
       [  32, 4247]], dtype=int64)

In [96]:
def output_lable(n):
    if n == 'Fake':
        return "Fake News"
    elif n == 'Real':
        return "Real News"
    
def manual_testing(news):
    input_data = [news]
    vectorized_input_data = tfvect.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    pred_LR = LR.predict(vectorized_input_data)
    pred_DT = DT.predict(vectorized_input_data)
    pred_GBC = GBC.predict(vectorized_input_data)
    pred_RFC = RFC.predict(vectorized_input_data)
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [97]:
news = str(input())
manual_testing(news)

MANCHESTER, N.H./STERLING HEIGHTS, MICHIGAN (Reuters) - As Donald Trump and Hillary Clinton criss-crossed the United States on Sunday in a final, frenzied burst of campaigning, the Republican said he was the â€œlast chanceâ€ to fix a broken country while his Democratic rival said a â€œmoment of reckoningâ€ had arrived. The two candidates in Tuesdayâ€™s presidential election presented starkly different views of the nation and evoked similarly disparate reactions from crowds gathered to hear them speak. In Iowa, Trump said he was the â€œlast chanceâ€ to fix immigration and trade. Outside Minneapolis, he said the Somali refugee population there was a â€œdisaster.â€ Supporters near Pittsburgh booed a song by musician Bruce Springsteen, who is set to campaign with Clinton on Monday night.  Chants of â€œlock her upâ€ came in waves even as news broke that the FBI again said Clinton should not face prosecution for her email practices while secretary of state. As rocker and guitarist Ted N